In [26]:
# Import the necessary packages.
import numpy as np
import pandas as pd
# https://pypi.org/project/names/ - used this name generator library.
import names

In [3]:
# Load the mtssamples_v1 dataset as a dataframe.
df = pd.read_csv("mtsamples_v1.csv")

In [4]:
# Display the dataframe.
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [5]:
# Drop the unnamed column.
df = df.drop('Unnamed: 0' , axis=1)

In [6]:
# Replace empty strings with NaN so that it gets filled up in the function below.
df = df.replace(r'^\s*$', np.nan, regex=True)

In [7]:
# Filter out the documents that do not have keywords.
filler = df[df['keywords'].isna()]

In [11]:
# Use KeyBERT to create keywords since close to 1000 docs don't have keywords.
from keybert import KeyBERT
model = KeyBERT('distilbert-base-nli-mean-tokens')

c:\Users\hp\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\hp\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--sentence-transformers--distilbert-base-nli-mean-tokens. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or t

In [11]:
# Testing keyBERT, getting satisfactory results, so proceeding to use it for all the rows.
keywords = model.extract_keywords(filler.iloc[0]['transcription'], top_n=10)

e:\Anaconda\Lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:403: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [12]:
# Find the keywords.
def find_keywords(row):
    return ", ".join([word[0] for word in model.extract_keywords(docs=row['transcription'], top_n=10)])


In [13]:
# Fill all keywords for each document that doesn't have keywords.
filler['keywords'] = filler.apply(find_keywords, axis = 1)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3140\1616112708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filler['keywords'] = filler.apply(find_keywords, axis = 1)


In [12]:
# Display dataframe.
filler.head()

,description,medical_specialty,sample_name,transcription,keywords
12,Cerebral Angiogram - moyamoya disease.,Neurology,Moyamoya Disease,"CC:, Confusion and slurred speech.,HX , (prima...",NaN
24,Blood in urine - Transitional cell cancer of ...,Urology,Urology Consut - 1,"CHIEF COMPLAINT:,",NaN
31,This is a 66-year-old male with signs and sym...,Urology,Urinary Retention,"CHIEF COMPLAINT:, Urinary retention.,HISTORY ...",NaN
32,Right distal ureteral calculus. The patient ...,Urology,Ureteral Calculus - Consult,"CHIEF COMPLAINT: , Right distal ureteral calcu...",NaN
39,The patient has a possibly torsion detorsion ...,Urology,Testicular Pain,"CHIEF COMPLAINT: , Testicular pain.,HISTORY OF...",NaN


In [13]:
# Create a copy of the DataFrame and reset the index for both df2 and filler2
df2 = df.copy()
df2 = df2.reset_index()
filler2 = filler.reset_index()

In [15]:
# merged DF, this has all the keywords present.
final_df = pd.merge(df2, filler2[['index', 'keywords']] , on='index', how='outer')

In [16]:
# Display final_df.
final_df.iloc[303]

index                                                              303
description           Total abdominal hysterectomy (TAH) and left s...
medical_specialty                                              Surgery
sample_name                               TAH & Salpingo-oophorectomy 
transcription        PREOPERATIVE DIAGNOSES:,1.  Hypermenorrhea.,2....
keywords_x                                                         NaN
keywords_y                                                         NaN
Name: 303, dtype: object

In [17]:
# Merge the keywords columns.
final_df['keywords'] = final_df['keywords_x'].combine_first(final_df['keywords_y'])

In [18]:
# Drop the duplicate columns and remove the index col as it's no longer needed.
final_df = final_df.drop(['keywords_x', 'keywords_y', 'index'], axis=1)

In [19]:
# Display final df head.
final_df.head()

,description,medical_specialty,sample_name,transcription,keywords
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [20]:
# Save final df as csv.
final_df.to_csv('filled_mtsamples.csv')

In [22]:
! pip install names

Using legacy 'setup.py install' for names, since package 'wheel' is not installed.
    Running setup.py install for names: started
    Running setup.py install for names: finished with status 'done'


You should consider upgrading via the 'C:\Users\hp\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [27]:
# Method to generate names.
def name_generator(row):
    return "NAME : " + names.get_full_name() + "  " + row['transcription']

In [29]:
# Applyting the generate names method to the dataframe.
final_df['transcription'] = final_df.apply(name_generator, axis=1)

In [30]:
# Save the final df as csv.
final_df.to_csv('synthetic_names_samples.csv')